In [1]:
from pathlib import Path
import os
import mne
import matplotlib.pyplot as plt
import numpy as np

from spectral.preproc import (
    load_data,
    zapline_clean,
    apply_pyprep,
)
from spectral.utils import ProjectPaths, print_timestamp, load_config

mne.viz.set_browser_backend("matplotlib")
# mne.viz.set_browser_backend("qt")
mne.set_config("MNE_BROWSER_THEME", "light")

# Initialize paths for your subject
subject_id = 101  # or whatever subject you're working with
paths = ProjectPaths(subject_id)

# Create all directories
paths.create_directories()

# This is analysis output, so it goes in the analysis folder
specparam_path = paths.analysis / "specparam"
specparam_path.mkdir(exist_ok=True)

# Print paths to verify
print_timestamp("Setting up project paths")
paths.show()

Using matplotlib as 2D backend.
Created directories for sub-101
Project root: /Users/daniel/PhD/Projects/psd-paths
Setting up project paths: 2025-07-15 01:09:07
────────────────────────────────────────────────────────────
Paths for sub-101
Project Root: /Users/daniel/PhD/Projects/psd-paths
────────────────────────────────────────────────────────────
  analysis     : /Users/daniel/PhD/Projects/psd-paths/data/derrivatives/sub-101/analysis
  data         : /Users/daniel/PhD/Projects/psd-paths/data/raw/sub-101
  epochs       : /Users/daniel/PhD/Projects/psd-paths/data/derrivatives/sub-101/epochs
  figures      : /Users/daniel/PhD/Projects/psd-paths/outputs/sub-101/figures
  logs         : /Users/daniel/PhD/Projects/psd-paths/data/derrivatives/sub-101/logs
  outputs      : /Users/daniel/PhD/Projects/psd-paths/outputs/sub-101
  preprocessed : /Users/daniel/PhD/Projects/psd-paths/data/derrivatives/sub-101/preprocessed
  processing   : /Users/daniel/PhD/Projects/psd-paths/data/derrivatives/sub

In [6]:
# Load the configuration
config = load_config()

# Access the bad channels list
bad_channels = config["preprocessing"]["channels_to_remove"]
print(f"Channels to remove: {bad_channels}")

fline = [50, 100]  # Line noise frequencies
h_freq = 40
l_freq = 1

filter_params = {
    "l_freq": l_freq,
    "h_freq":h_freq,
    "h_trans_bandwidth": "auto",
    "fir_window": "hamming",
    "fir_design": "firwin",
    "phase": "zero",
    "picks": ["ecg", "eeg"],
}

raw = load_data(subject_id, data_path=paths.data).resample(
    250, method="polyphase", verbose=True
)
total_duration = raw.times[-1]

raw_filtered = (
    raw.copy()
    .resample(250, method="polyphase", verbose=True)
    .drop_channels(bad_channels)
    .notch_filter(freqs=[fline], method="fir",   picks=['eeg', 'ecg'])
    .filter(**filter_params)
    .crop(tmin=3.0, tmax=total_duration - 3, include_tmax=True)
)


from spectral.artifacts import detect_bad_channels_comprehensive, detect_bad_segments_adaptive,identify_and_handle_reference_channel

raw = identify_and_handle_reference_channel(raw)
raw.info["bads"]

Channels to remove: ['E67', 'E73', 'E82', 'E91', 'E92', 'E102', 'E111', 'E120', 'E133', 'E145', 'E165', 'E174', 'E187', 'E199', 'E208', 'E209', 'E216', 'E217', 'E218', 'E219', 'E225', 'E226', 'E227', 'E228', 'E229', 'E230', 'E231', 'E232', 'E233', 'E234', 'E235', 'E236', 'E237', 'E238', 'E239', 'E240', 'E241', 'E242', 'E243', 'E244', 'E245', 'E246', 'E247', 'E248', 'E249', 'E250', 'E251', 'E252', 'E253', 'E254', 'E255', 'E256']
Loading data from: /Users/daniel/PhD/Projects/psd-paths/data/raw/sub-101/ses-01/eeg/sub-101_ses-01_task-rest_eeg.set
Loaded 258 channels, 300.4 seconds of data
Identified ECG channels: ['ECG']
Removed 'VREF' channel.
Applying GSN-HydroCel-256 montage...
Polyphase resampling neighborhood: ±2 input samples
Sampling frequency of the instance is already 250.0, returning unmodified.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowe

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 205 out of 205 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s



=== Checking for reference channel issues ===


[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 205 out of 205 | elapsed:    0.3s finished


[]

In [ ]:
# Step 2: Remove line noise (before other processing)

line_freq = 50.0
low_freq = 1.0

high_freq = 40.0,
print("\n=== Step 2: Removing line noise ===")
raw_zapped = zapline_clean(
    raw, 
    fline=line_freq, 
    ntimes=2,  # Conservative
    method='line'
)

# Step 3: Detect bad channels (on minimally processed data)
print("\n=== Step 3: Detecting bad channels ===")
bad_channels, bad_channels_by_method = detect_bad_channels_comprehensive(
    raw_zapped,
    use_pyprep=True,
    use_autoreject_epochs=True
)
    

NameError: name 'raw' is not defined

In [ ]:

from spectral.artifacts import detect_bad_channels_comprehensive, detect_bad_segments_adaptive

preprocessing_info = {
    'subject_id': subject_id,
    'original_n_channels': len(raw.ch_names),
    'original_duration': raw.times[-1]
}
# Step 3: Detect bad channels (on minimally processed data)
print("\n=== Step 3: Detecting bad channels ===")
bad_channels, bad_channels_by_method = detect_bad_channels_comprehensive(
    raw_zapped,
    use_pyprep=True,
    use_autoreject_epochs=True
)

raw_step3 = raw_zapped.copy()
raw_step3.info['bads'] = bad_channels
preprocessing_info['bad_channels'] = bad_channels
preprocessing_info['bad_channels_by_method'] = bad_channels_by_method

# Step 4: Detect bad segments (before filtering)
print("\n=== Step 4: Detecting bad segments ===")
bad_annotations = detect_bad_segments_adaptive(
    raw_step3,
    bad_channels=bad_channels,
    epoch_length=1.0,
    overlap=0.5
)

raw_step4 = raw_step3.copy()
raw_step4.set_annotations(raw_step4.annotations + bad_annotations)
preprocessing_info['n_bad_segments'] = len(bad_annotations)
preprocessing_info['bad_time_percent'] = (
    sum(bad_annotations.duration) / raw.times[-1] * 100
)
